In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import models
import numpy as np
import argparse
import time
import sys

In [ ]:
tf.config.experimental.list_physical_devices('GPU')

In [ ]:
!nvidia-smi

In [ ]:
batch_size_per_gpu = 256
epochs = 5
n_gpus = 1

batch_size = batch_size_per_gpu * n_gpus

In [ ]:
# get the training dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

device_type = "GPU"

devices = tf.config.experimental.list_physical_devices(device_type)

devices_names = [d.name.split("e:")[1] for d in devices]

print(f"{device_type} available: {devices_names}")

# create a training strategy for utilizing the GPUs
strategy = tf.distribute.MirroredStrategy(devices=devices_names[:n_gpus])

with strategy.scope():
    # load a model architecture from resnet_v2
    model = tf.keras.applications.resnet_v2.ResNet50V2(
        include_top=True, weights=None, input_shape=(32, 32, 3), classes=10
    )
    opt = tf.keras.optimizers.SGD(0.01 * n_gpus)
    model.compile(
        loss="sparse_categorical_crossentropy", optimizer=opt, metrics=["accuracy"]
    )

In [ ]:
model.summary()

In [ ]:
model.fit(x=x_train, y=y_train, batch_size=batch_size, epochs=epochs, verbose=2)